FORWARD SCAN & SONAR DATASET

In [89]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score,RandomizedSearchCV,StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE, RandomOverSampler

# DATA COLLECTION & PREPROCESSOR

In [90]:
BNG_sonar = pd.read_csv('/content/BNG_SONAR.csv', header=None)



In [91]:
BNG_sonar.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.020908,0.068070,0.004803,0.021266,0.060084,0.129593,0.084767,0.105890,0.206086,0.250013,...,0.015695,0.009727,0.006086,0.006371,0.005380,0.001661,0.007355,0.005110,0.008948,Mine
1,0.034281,0.040975,0.038144,0.141582,0.043993,0.028720,0.066098,0.059756,0.175466,0.230111,...,0.002345,0.015357,0.002124,0.007650,0.002820,0.020329,0.014374,0.005052,0.009116,Mine
2,0.008932,0.014497,0.020886,0.068266,0.011985,0.046423,0.150748,0.104827,0.121276,0.134928,...,0.016220,0.012289,0.021733,0.011117,0.000481,0.003959,0.003542,0.005403,0.001863,Rock
3,0.068667,0.048465,0.034146,0.047959,0.099094,0.118204,0.105556,0.246807,0.426913,0.497205,...,0.008931,0.008403,0.012409,0.004066,0.023306,0.015380,0.003582,0.004866,0.001443,Mine
4,0.006277,0.010476,0.008242,0.013835,0.019348,0.069207,0.084286,0.080349,0.158213,0.142696,...,0.003515,0.013871,0.012691,0.014288,0.006432,0.005682,0.007099,0.007791,0.017659,Rock


In [92]:
BNG_sonar.shape

(5000, 61)

In [93]:
BNG_sonar.describe()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.029432,0.037750,0.042523,0.052061,0.072424,0.101267,0.120254,0.133879,0.175672,0.203712,...,0.016006,0.013000,0.010515,0.011045,0.009405,0.008350,0.007671,0.007881,0.007983,0.006414
std,0.022844,0.032590,0.037100,0.044866,0.054170,0.057533,0.060690,0.081900,0.115106,0.128296,...,0.011917,0.009317,0.006931,0.007338,0.007122,0.005844,0.005755,0.006507,0.006181,0.004937
min,-0.017166,-0.035656,-0.035673,-0.062108,-0.015298,-0.010949,-0.011921,-0.036867,-0.016799,-0.019694,...,-0.014395,-0.000298,-0.000935,-0.000854,-0.004038,-0.002632,-0.000341,-0.003467,-0.000830,-0.007313
25%,0.014528,0.016333,0.019158,0.024885,0.037708,0.066193,0.082784,0.083289,0.101368,0.117152,...,0.008453,0.007058,0.005302,0.005794,0.004402,0.004498,0.003720,0.003640,0.003958,0.003092
50%,0.023190,0.029566,0.033156,0.041492,0.059715,0.089801,0.109513,0.114104,0.149585,0.181667,...,0.013791,0.010705,0.009052,0.009414,0.007494,0.007000,0.006053,0.005864,0.006389,0.005197
75%,0.034463,0.046575,0.052392,0.061384,0.090646,0.128017,0.149883,0.164021,0.216917,0.253034,...,0.019695,0.015791,0.013926,0.013997,0.011659,0.010404,0.009796,0.009902,0.009679,0.007968
max,0.160354,0.197399,0.265992,0.309589,0.356305,0.342328,0.394941,0.520403,0.751314,0.829719,...,0.081259,0.058290,0.037323,0.045771,0.041579,0.037837,0.037507,0.039059,0.038891,0.032327


In [94]:
BNG_sonar[60].value_counts()

,count
60,
Mine,2690
Rock,2309
0.009318,1


In [95]:
BNG_sonar.groupby(60).mean()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
60,,,,,,,,,,,,,,,,,,,,,
0.009318,0.009318,0.009318,0.009318,0.009318,0.009318,0.009318,0.009318,0.009318,0.009318,0.009318,...,0.009318,0.009318,0.009318,0.009318,0.009318,0.009318,0.009318,0.009318,0.009318,0.009318
Mine,0.034649,0.042066,0.046437,0.058703,0.077736,0.107298,0.123826,0.144965,0.203715,0.236730,...,0.016617,0.013245,0.010617,0.011960,0.009290,0.008762,0.007681,0.008057,0.008231,0.006514
Rock,0.023363,0.032734,0.037977,0.044341,0.066262,0.094280,0.116142,0.121016,0.143073,0.165330,...,0.015297,0.012716,0.010395,0.009980,0.009539,0.007869,0.007659,0.007676,0.007695,0.006297


In [96]:
# Features and target
x = BNG_sonar.iloc[:, :-1]  # Features
y = BNG_sonar.iloc[:, -1]   # Target

In [97]:
print(x)
print(y)

            0         1         2         3         4         5         6   \
0     0.020908  0.068070  0.004803  0.021266  0.060084  0.129593  0.084767   
1     0.034281  0.040975  0.038144  0.141582  0.043993  0.028720  0.066098   
2     0.008932  0.014497  0.020886  0.068266  0.011985  0.046423  0.150748   
3     0.068667  0.048465  0.034146  0.047959  0.099094  0.118204  0.105556   
4     0.006277  0.010476  0.008242  0.013835  0.019348  0.069207  0.084286   
...        ...       ...       ...       ...       ...       ...       ...   
4995  0.011962  0.015225  0.007808  0.035554  0.169388  0.180563  0.152202   
4996  0.015319  0.019144  0.041496  0.066714  0.041219  0.089354  0.100451   
4997  0.039620  0.005314  0.016893  0.046773  0.061555  0.080277  0.173763   
4998  0.021974  0.014970  0.038422  0.038137  0.049918  0.123267  0.118368   
4999  0.010184  0.009845 -0.001675  0.021164  0.020999  0.100614  0.149316   

            7         8         9   ...        50        51    

In [98]:
# Feature Scaling
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Oversampling
ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
x_resampled, y_resampled = ros.fit_resample(x_scaled, y)

# TRAINING & TESTING

In [108]:
# Split the data
x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42)


In [112]:
# Hyperparameter tuning for LDA
lda = LinearDiscriminantAnalysis()
param_grid = [
    {'solver': ['svd'], 'shrinkage': [None]},  # Only 'None' for 'svd'
    {'solver': ['lsqr', 'eigen'], 'shrinkage': [None, 'auto']}  # Options for 'lsqr' and 'eigen'
]

In [113]:
# Set up RandomizedSearchCV
grid_search_lda = RandomizedSearchCV(lda, param_grid, n_iter=len(param_grid), cv=5, scoring='accuracy')
grid_search_lda.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=LinearDiscriminantAnalysis(), n_iter=2,
                   param_distributions=[{'shrinkage': [None],
                                         'solver': ['svd']},
                                        {'shrinkage': [None, 'auto'],
                                         'solver': ['lsqr', 'eigen']}],
                   scoring='accuracy')

In [114]:

print("Best Parameters:", grid_search_lda.best_params_)

Best Parameters: {'solver': 'lsqr', 'shrinkage': 'auto'}


In [115]:
# Hyperparameter tuning for Random Forest
rf = RandomForestClassifier()
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
grid_search_rf = RandomizedSearchCV(rf, param_grid_rf, n_iter=10, cv=5, scoring='accuracy')
grid_search_rf.fit(x_train, y_train)
best_rf = grid_search_rf.best_estimator_

In [116]:
print(x.shape, x_train.shape, x_test.shape)


(5000, 60) (6456, 60) (1614, 60)


In [117]:
print(x_train)
print(y_train)

[[ 2.57263577  2.04071452  2.88627138 ...  0.4273586  -0.0215804
  -0.91992937]
 [-0.99312015 -0.92894784 -0.80414449 ...  0.71277486  1.61665443
   0.14276592]
 [-1.06771957 -0.0265078   0.17602535 ... -0.15208097  0.08941507
  -0.10680339]
 ...
 [ 1.83587895 -0.62858457 -0.43869353 ... -0.00437767 -0.59144342
   0.53494627]
 [ 0.75703301  3.10518135 -0.63485689 ...  0.51004786  0.01563382
  -0.17385488]
 [ 1.79936726  0.22414272  0.34911384 ... -0.50758329 -0.1045843
  -0.15744651]]
1088        Mine
1938        Rock
2702        Rock
8019        Rock
1327        Mine
          ...   
7058    0.009318
4813        Rock
3225        Rock
4597        Mine
1842        Mine
Name: 60, Length: 6456, dtype: object


# MODEL TRAINING -> LOGISTIC REGRESSION

In [118]:
# Standardize the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [120]:
# Model Stacking with LDA and XGBoost
base_models = [
    ('lda', best_lda),
    ('logistic', LogisticRegression(max_iter=1000, C=1.0, penalty='l2')),
    ('random_forest', best_rf),
    ('xgboost', XGBClassifier(eval_metric='logloss'))
]
stack_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())
stack_model.fit(x_train, y_train)

StackingClassifier(estimators=[('lda',
                                LinearDiscriminantAnalysis(shrinkage='auto',
                                                           solver='lsqr')),
                               ('logistic', LogisticRegression(max_iter=1000)),
                               ('random_forest',
                                RandomForestClassifier(min_samples_split=10,
                                                       n_estimators=300)),
                               ('xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_b...
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...))],
                   final_estimator=LogisticRegression())

In [121]:
# Cross-Validation with Stratified K-Folds
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = []

for train_index, val_index in skf.split(x_train, y_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    stack_model.fit(X_train_fold, y_train_fold)
    fold_score = stack_model.score(X_val_fold, y_val_fold)
    cv_scores.append(fold_score)

In [122]:
print(f'Stratified Cross-Validation Accuracy: {np.mean(cv_scores)}')

Stratified Cross-Validation Accuracy: 0.9355633954928362


# MODEL EVALUATION

In [123]:
model = LogisticRegression(penalty='l2', C=0.01, max_iter=1000)  # Adjust C for regularization strength
model.fit(x_train, y_train)

LogisticRegression(C=0.01, max_iter=1000)

In [124]:
train_predictions = stack_model.predict(x_train)
test_predictions = stack_model.predict(x_test)

In [125]:
train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)

In [126]:
print(f'Train Accuracy: {train_accuracy}')
print(f'Test Accuracy: {test_accuracy}')

Train Accuracy: 0.9947335811648079
Test Accuracy: 0.9275092936802974


In [ ]:
# Confusion Matrix and Classification Report
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

[[538   0   0]
 [  0 475  63]
 [  0  64 474]]
              precision    recall  f1-score   support

    0.009318       1.00      1.00      1.00       538
        Mine       0.88      0.88      0.88       538
        Rock       0.88      0.88      0.88       538

    accuracy                           0.92      1614
   macro avg       0.92      0.92      0.92      1614
weighted avg       0.92      0.92      0.92      1614



In [ ]:
input_data = (0.0307, 0.0523, 0.0653, 0.0521, 0.0611, 0.0577, 0.0665, 0.0664, 0.1460, 0.2792,
              0.3877, 0.4992, 0.4981, 0.4972, 0.5607, 0.7339, 0.8230, 0.9173, 0.9975, 0.9911,
              0.8240, 0.6498, 0.5980, 0.4862, 0.3150, 0.1543, 0.0989, 0.0284, 0.1008, 0.2636,
              0.2694, 0.2930, 0.2925, 0.3998, 0.3660, 0.3172, 0.4609, 0.4374, 0.1820, 0.3376,
              0.6202, 0.4448, 0.1863, 0.1420, 0.0589, 0.0576, 0.0672, 0.0269, 0.0245, 0.0190,
              0.0063, 0.0321, 0.0189, 0.0137, 0.0277, 0.0152, 0.0052, 0.0121, 0.0124, 0.0055)

input_data_as_numpy_array = np.asarray(input_data)
input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)
input_data_scaled = scaler.transform(input_data_reshaped)
prediction = stack_model.predict(input_data_scaled)
print(prediction)
print('The object is a Rock' if prediction[0] == 'Rock' else 'The object is a Mine')


['Mine']
The object is a Mine
